In [4]:
import pandas as pd
import datatable as dt
import numpy as np

train = dt.fread("/input/riiid-test-answer-prediction/train.csv").to_pandas()


# **Generating same feature files minus the validation**

In [9]:
val_idx = pd.read_pickle("/home/dfs/cv1_valid.pickle")

In [3]:
train = train[~train.index.isin(val_idx.row_id)]

In [5]:
#Convert users into sequences
groups = train[['user_id', 'content_id', 'answered_correctly']].groupby('user_id').apply(lambda r: (
            r['content_id'].values,
            r['answered_correctly'].values))

In [6]:
groups = groups.apply(lambda x: x[1][-140:].tolist())
groups = groups.to_dict()

In [7]:
lec_data = pd.read_csv("/input/riiid-test-answer-prediction/lectures.csv")
lec_dict = lec_data[["lecture_id", "tag"]].set_index("lecture_id").tag.to_dict()

In [8]:
que_data = pd.read_pickle("/home/dfs/que_data.pickle")
questions = que_data.to_dict("index")

In [9]:
def gen_lec_data(train):
    
    train = train[train["content_type_id"] == True]
    
    train = train.groupby("user_id").apply(lambda x: x.content_id.tolist())
    train = pd.DataFrame(train)
    train.columns = ["CUM_CONCAT"]
    
    global lec_history
    lec_history = {}

    def f(row):

        global lec_history
        lec_history[str(row.name)] = {}

        for i in row.CUM_CONCAT:
            lec_tag  = lec_dict[i]
            lec_history[str(row.name)][str(lec_tag)] = 1

        return row

    train.apply(f, axis=1)
    
    return lec_history
    
watched_tags = gen_lec_data(train)

In [10]:
#With lectures
timetable = train.groupby("user_id")[["user_id","timestamp"]].tail(5).groupby("user_id")["timestamp"].apply(lambda x: x.values)

In [11]:
timetable_df = pd.DataFrame(columns=["first_timestamp","second_timestamp","third_timestamp","fourth_timestamp","fifth_timestamp"], index=timetable.index)

In [12]:
timetable_df["fifth_timestamp"] = timetable.apply(lambda x: x[0])
timetable_df["fourth_timestamp"] = timetable.apply(lambda x: x[1] if len(x) > 1 else 0)
timetable_df["third_timestamp"] = timetable.apply(lambda x: x[2] if len(x) > 2 else 0)
timetable_df["second_timestamp"] = timetable.apply(lambda x: x[3] if len(x) > 3 else 0)
timetable_df["first_timestamp"] = timetable.apply(lambda x: x[4] if len(x) > 4 else 0)

In [13]:
timetable_df = timetable_df/8.64e+7

In [14]:
train["time_diff"] = train.groupby("user_id")["timestamp"].diff().fillna(0).astype("float32")
train["ts_diff_shifted"] = train.groupby("user_id")["time_diff"].shift(1).replace(0, np.nan)
train["ts_diff_shifted"] = train.groupby("user_id")["ts_diff_shifted"].fillna(method="ffill").fillna(0)

In [15]:
train["time_diff1"] = train.groupby("user_id")["timestamp"].diff(2).fillna(0).astype("float32")
train["ts_diff_shifted_2"] = train.groupby("user_id")["time_diff1"].shift(1).replace(0, np.nan)
train["ts_diff_shifted_2"] = train.groupby("user_id")["ts_diff_shifted_2"].fillna(method="ffill").fillna(0)

In [16]:
ts_diff_shifted = train.groupby("user_id")["ts_diff_shifted"].last()
ts_diff_shifted2 = train.groupby("user_id")["ts_diff_shifted_2"].last()

In [17]:
ts_diff_shifted = pd.DataFrame(ts_diff_shifted)
ts_diff_shifted.columns = ["ts_diff_shifted"]

ts_diff_shifted2 = pd.DataFrame(ts_diff_shifted2)
ts_diff_shifted = ts_diff_shifted.join(ts_diff_shifted2, how="left")
ts_diff_shifted.columns = ["ts_diff_shifted","ts_diff_shifted_2"]

In [18]:
lecture_count = train.groupby("user_id")["content_type_id"].sum()
interactions = train.groupby("user_id")["content_type_id"].count()


base = pd.DataFrame(lecture_count)
base.columns = ["lecs_n"]

base = base.join(interactions, how="left")
base.columns = ["lecs_n", "interaction_n"]

In [19]:
base = base.join(ts_diff_shifted)

In [20]:
train["timestamp"] = train["timestamp"]/8.64e+7
train["time_diff"] = train.groupby("user_id")["timestamp"].diff().fillna(0).astype("float32")

In [21]:
train["sessions"] = train["time_diff"] > 0.083
sessions = train.groupby("user_id")["sessions"].sum()

base = base.join(sessions)
base = base.rename(columns={"user_id":"session"})


In [22]:
train["sessions"] = train.groupby("user_id")["sessions"].cumsum()

train["session_count"] =1
train["session_count"] = train.groupby(["user_id","sessions"])["session_count"].cumsum()

In [23]:
session_count = train.groupby("user_id")[["session_count", "user_id"]].tail(1).set_index("user_id", drop=True)
base = base.join(session_count)

del train["session_count"]
del train["sessions"]

In [24]:
train["mean_pause"] = ((train.time_diff > 0.083)*train.time_diff).replace(0,np.nan)
test = train[["mean_pause","user_id"]]
test = test.dropna()

base = base.join(test.groupby("user_id")["mean_pause"].sum())
base = base.rename(columns={"mean_pause":"sum_pauses"})
base.sum_pauses = base.sum_pauses.fillna(0)

In [25]:
train = train[train["content_type_id"] == False]

In [26]:
train.prior_question_had_explanation = train.prior_question_had_explanation.fillna(False).astype("bool")
train.prior_question_elapsed_time = train.prior_question_elapsed_time.fillna(0)

In [27]:
base = base.join(train.groupby("user_id")["prior_question_had_explanation"].sum())
base = base.rename(columns={"prior_question_had_explanation":"had_exp"})

In [28]:
train = train.merge(que_data[["part"]], how="left", right_index=True, left_on="content_id")
train.part = train.part.astype("int8")

In [34]:
train["last_part"] = train.groupby("user_id")["part"].shift().fillna(-1).astype("int8")

In [46]:
a = train.groupby("user_id")[["user_id","last_part"]].tail(1).set_index("user_id", drop=True)
base = base.join(pd.DataFrame(a, columns=["last_part"]))

In [47]:
base

,lecs_n,interaction_n,ts_diff_shifted,ts_diff_shifted_2,sessions,session_count,sum_pauses,had_exp,last_part
user_id,,,,,,,,,
115,0,46,118231.0,118231.0,1,7,7.712886,6,4
124,0,30,16819.0,16819.0,0,30,0.000000,0,7
2746,1,20,38370.0,65141.0,0,20,0.000000,11,2
5382,3,128,269227.0,432686.0,16,1,23.964521,113,5
8623,3,112,43466.0,101956.0,10,10,9.781732,96,2
...,...,...,...,...,...,...,...,...,...
2147470770,2,228,40994.0,74437.0,14,20,32.620239,195,5
2147470777,6,640,420113.0,420113.0,11,85,151.120880,603,6
2147481750,0,50,27588.0,53729.0,1,20,0.597832,39,5


In [48]:
def f(x):
    
    parts = x.part.values.tolist()
    nb = x.answered_correctly.tolist()
    ret = [0]*7
    
    for i, k in enumerate(parts):
        ret[k-1] = nb[i]
        
    return ret

In [49]:
a = pd.DataFrame(train.groupby(["user_id","part"])["answered_correctly"].count())

a["user_id"] = a.index.get_level_values(0)
a["part"] = a.index.get_level_values(1)

a.reset_index(drop=True, inplace=True)
base = base.join(pd.DataFrame(a.groupby("user_id").apply(f), columns=["part_count"]))

In [50]:
a = pd.DataFrame(train.groupby(["user_id","part"])["answered_correctly"].sum())

a["user_id"] = a.index.get_level_values(0)
a["part"] = a.index.get_level_values(1)

a.reset_index(drop=True, inplace=True)
base = base.join(pd.DataFrame(a.groupby("user_id").apply(f), columns=["part_corr"]))

In [51]:
tmp_train = pd.read_pickle("/home/dfs/data_2")

In [52]:
tmp_train = tmp_train[["k", "content_id"]]

In [53]:
que_2_k = tmp_train.groupby("content_id")["k"].first().to_dict()

del tmp_train

In [54]:
que_2_k = pd.DataFrame(pd.Series(que_2_k), columns=["k"])

In [55]:
train = train.merge(que_2_k, how="left", left_on="content_id", right_index=True)

In [56]:
def f(x):
    
    k = x.k.values.tolist()
    nb = x.answered_correctly.tolist()
    ret = [0]*20
    
    for i, j in enumerate(k):
        ret[j] = nb[i]
        
    return ret

In [57]:
a = pd.DataFrame(train.groupby(["user_id", "k"])["answered_correctly"].count())

a["user_id"] = a.index.get_level_values(0)
a["k"] = a.index.get_level_values(1)

a.reset_index(drop=True, inplace=True)

k_count = pd.DataFrame(a.groupby("user_id").apply(f), columns=["k_count"])

In [58]:
a = pd.DataFrame(train.groupby(["user_id", "k"])["answered_correctly"].sum())

a["user_id"] = a.index.get_level_values(0)
a["k"] = a.index.get_level_values(1)

a.reset_index(drop=True, inplace=True)

k_corr = pd.DataFrame(a.groupby("user_id").apply(f), columns=["k_corr"])

In [59]:
base = base.join(k_corr).join(k_count)
del k_corr
del k_count

In [60]:
prior_part_mean_dict = {1: 22166.159642501425,
 2: 18714.69673913695,
 3: 23620.317746179924,
 4: 23762.753651169547,
 5: 25094.620302855932,
 6: 32417.37918735745,
 7: 47444.16407400242}

In [61]:
a = pd.DataFrame(train.groupby(["user_id","part"])["prior_question_elapsed_time"].sum())

a["user_id"] = a.index.get_level_values(0)
a["part"] = a.index.get_level_values(1)

a.reset_index(drop=True, inplace=True)

In [62]:
def f(x):
    
    parts = x.part.values.tolist()
    nb = x.prior_question_elapsed_time.tolist()
    ret = [0]*7
    
    for i, k in enumerate(parts):
        ret[k-1] = nb[i]
        
    return ret

part_et = pd.DataFrame(a.groupby("user_id").apply(f), columns=["part_et"])

In [63]:
a = pd.DataFrame(train.groupby(["user_id","part"])["prior_question_had_explanation"].sum())

a["user_id"] = a.index.get_level_values(0)
a["part"] = a.index.get_level_values(1)

a.reset_index(drop=True, inplace=True)


def f(x):
    
    parts = x.part.values.tolist()
    nb = x.prior_question_had_explanation.tolist()
    ret = [0]*7
    
    for i, k in enumerate(parts):
        ret[k-1] = nb[i]
        
    return ret

part_explan = pd.DataFrame(a.groupby("user_id").apply(f), columns=["part_explan"])

In [64]:
a = train.groupby("user_id")[["user_id","prior_question_had_explanation"]].tail(5)

In [65]:
priors_5 = pd.DataFrame(a.groupby("user_id")["prior_question_had_explanation"].apply(lambda x: x.tolist()))
priors_5.columns = ["priors_5"]

In [66]:
base = base.join(part_et).join(part_explan).join(priors_5)

In [67]:
last_ts = train[train.answered_correctly == True].groupby("user_id")["timestamp"].last()

last_ts = pd.DataFrame(last_ts)
last_ts.columns = ["recent_corr"]

In [68]:
el_sum = train.groupby("user_id")["prior_question_elapsed_time"].sum()

In [69]:
el_sum = pd.DataFrame(el_sum)
el_sum.columns = ["el_sum"]

In [70]:
base = base.join(el_sum).join(last_ts)

In [71]:
a = train.groupby("content_id")["answered_correctly"].agg(["count","mean"])

easy_questions = a[(a["mean"] > 0.8)]
easy_questions["easy_question"] = True

del easy_questions["mean"]
del easy_questions["count"]


hard_questions = a[(a["count"] > 100) & (a["mean"] < 0.4)]
hard_questions["hard_question"] = True

del hard_questions["mean"]
del hard_questions["count"]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [72]:
hardest = train[train.content_id.isin(hard_questions.index)].groupby("user_id")["answered_correctly"].agg(["count","sum"])
hardest.columns = ["hard_ct", "hard_cr"]

In [73]:
hardest = train[train.content_id.isin(hard_questions.index)].groupby("user_id")["answered_correctly"].agg(["count","sum"])
hardest.columns = ["hard_ct", "hard_cr"]
base = base.join(hardest)

base.hard_ct = base.hard_ct.fillna(0).astype("int16")
base.hard_cr = base.hard_cr.fillna(0).astype("int16")

del hardest

In [74]:
easiest = train[train.content_id.isin(easy_questions.index)].groupby("user_id")["answered_correctly"].agg(["count","sum"])
easiest.columns = ["easy_ct", "easy_cr"]
base = base.join(easiest)

base.easy_ct = base.easy_ct.fillna(0).astype("int16")
base.easy_cr = base.easy_cr.fillna(0).astype("int16")

del easiest

In [75]:
"""Dictionary containing accuracy, number of questions and number of correct questions and last lecture"""

summ = train.groupby("user_id")[["answered_correctly"]].sum()
mean = train.groupby("user_id")[["answered_correctly"]].mean()
count = train.groupby("user_id")[["answered_correctly"]].count()

last_lecs = train[["user_id", "content_id"]].groupby("user_id").tail(1)
last_lecs.index = last_lecs.user_id
del last_lecs["user_id"]



user_info = summ.join(mean, how="left", rsuffix="_mean").join(count, how="left", rsuffix="_count").join(last_lecs, how="left", rsuffix="_lecs").join(timetable_df, how="left")
user_info.columns = ["correct_count", "mean_acc", "count", "last_lec","first_timestamp","second_timestamp", "third_timestamp","fourth_timestamp", "fifth_timestamp"]
user_info["tmp"] = 0


user_info.last_lec = user_info.last_lec.fillna(0)

user_info.mean_acc = user_info.mean_acc.astype("float16")
user_info.correct_count = user_info.correct_count.astype("int32")

In [76]:
user_info = user_info.join(base)
user_info = user_info.to_dict("index")

In [77]:
user_info[115]

{'correct_count': 32,
 'mean_acc': 0.69580078125,
 'count': 46,
 'last_lec': 3364,
 'first_timestamp': 7.732523645833333,
 'second_timestamp': 7.732523645833333,
 'third_timestamp': 7.732523645833333,
 'fourth_timestamp': 7.731155231481481,
 'fifth_timestamp': 7.731155231481481,
 'tmp': 0,
 'lecs_n': 0,
 'interaction_n': 46,
 'ts_diff_shifted': 118231.0,
 'ts_diff_shifted_2': 118231.0,
 'sessions': 1,
 'session_count': 7,
 'sum_pauses': 7.712886333465576,
 'had_exp': 6,
 'last_part': 4,
 'part_count': [37, 1, 3, 3, 2, 0, 0],
 'part_corr': [26, 1, 2, 1, 2, 0, 0],
 'k_corr': [2, 19, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 6, 0, 0, 0, 0, 0, 2, 0],
 'k_count': [3, 25, 0, 0, 4, 2, 0, 0, 0, 1, 0, 0, 8, 0, 0, 0, 0, 0, 3, 0],
 'part_et': [745000.0, 21000.0, 51000.0, 42999.0, 37000.0, 0, 0],
 'part_explan': [6, 0, 0, 0, 0, 0, 0],
 'priors_5': [False, False, False, False, False],
 'el_sum': 896999.0,
 'recent_corr': 7.732523645833333,
 'hard_ct': 6,
 'hard_cr': 1,
 'easy_ct': 20,
 'easy_cr': 16}

In [78]:
containers_mean = train.groupby("task_container_id")["answered_correctly"].mean()
containers_mean = containers_mean.to_dict()

In [79]:
hard_questions = hard_questions["hard_question"].to_dict()
easy_questions = easy_questions["easy_question"].to_dict()

In [80]:
repeated_que_count = train.groupby(["user_id", "content_id"])[["answered_correctly"]].count()

def f(x):
    return x.droplevel(0).answered_correctly.to_dict()

repeated_que_count = repeated_que_count.groupby(level=0).apply(f).to_dict()

In [5]:
del train
train = dt.fread("/input/riiid-test-answer-prediction/train.csv").to_pandas()
train = train[train.content_type_id == False]

In [6]:
heat = pd.read_pickle("/home/LGBM build and test/heat_pickle")

In [7]:
nlargest_que =train.groupby("content_id")["user_id"].count().nlargest(300).index

In [16]:
#train = train[~train.index.isin(val_idx.row_id)]
train_value = train[["user_id","content_id","answered_correctly","task_container_id"]].values

In [17]:
top_que_history = {}

In [18]:
from tqdm import tqdm

for i in tqdm(range(train_value.shape[0])):

    que_id = train_value[i, 1]
    user_id = train_value[i, 0]
    correct = train_value[i, 2]

    if que_id in nlargest_que:

        if correct == True: 

            if top_que_history.get(user_id, -1) == -1:
                top_que_history[user_id] = {}
            top_que_history[user_id][que_id] = True


100%|██████████| 96817414/96817414 [07:33<00:00, 213572.52it/s]


In [19]:
cols = []
for i in nlargest_que:
    cols.append("q_"+str(i))

In [20]:
que_corr = {}

for i in cols:
    dct = heat[i].nlargest(6)[1:].to_dict()
    que_corr[int(i[2:])] = {int(j[2:]):dct[j] for j in dct}

# **Import the model, test re-creating the features during inference**

In [ ]:
train = dt.fread("/input/riiid-test-answer-prediction/train.csv").to_pandas()
val_idx = pd.read_pickle("/home/dfs/cv1_valid.pickle")

In [ ]:
train.prior_question_elapsed_time = train.prior_question_elapsed_time.fillna(0)
a = train.groupby("user_id")["answered_correctly"].count()

prob_users = train[(train.timestamp == 0) & (train.prior_question_elapsed_time != 0)].user_id.unique().tolist()
prob_users.extend([171953341,1958652827])  #These two users only answer to one bundle
prob_users.extend(a[a == 1].index.tolist())
prob_users = list(set(prob_users))

del a

In [ ]:
train = train[~train.user_id.isin(prob_users)]
validation = train[train.index.isin(val_idx.row_id)]

del train

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
tested_val = pd.read_pickle("validation_df")

In [ ]:
for u in user_info:
    if not u in top_que_history:
        top_que_history[u] = {}

In [ ]:
for u in user_info:
    user_info[u]["count_2"] = user_info[u]["count"]
    user_info[u]["part_count_2"] = user_info[u]["part_count"].copy()

In [ ]:
#https://www.kaggle.com/its7171/cv-strategy
class Iter_Valid(object):
    def __init__(self, df, max_user=1000):
        df = df.reset_index(drop=True)
        self.df = df
        self.user_answer = df['user_answer'].astype(str).values
        self.answered_correctly = df['answered_correctly'].astype(str).values
        df['prior_group_responses'] = "[]"
        df['prior_group_answers_correct'] = "[]"
        self.sample_df = df[df['content_type_id'] == 0][['row_id']]
        self.sample_df['answered_correctly'] = 0
        self.len = len(df)
        self.user_id = df.user_id.values
        self.task_container_id = df.task_container_id.values
        self.content_type_id = df.content_type_id.values
        self.max_user = max_user
        self.current = 0
        self.pre_user_answer_list = []
        self.pre_answered_correctly_list = []

    def __iter__(self):
        return self
    
    def fix_df(self, user_answer_list, answered_correctly_list, pre_start):
        df= self.df[pre_start:self.current].copy()
        sample_df = self.sample_df[pre_start:self.current].copy()
        df.loc[pre_start,'prior_group_responses'] = '[' + ",".join(self.pre_user_answer_list) + ']'
        df.loc[pre_start,'prior_group_answers_correct'] = '[' + ",".join(self.pre_answered_correctly_list) + ']'
        self.pre_user_answer_list = user_answer_list
        self.pre_answered_correctly_list = answered_correctly_list
        return df, sample_df

    def __next__(self):
        added_user = set()
        pre_start = self.current
        pre_added_user = -1
        pre_task_container_id = -1

        user_answer_list = []
        answered_correctly_list = []
        while self.current < self.len:
            crr_user_id = self.user_id[self.current]
            crr_task_container_id = self.task_container_id[self.current]
            crr_content_type_id = self.content_type_id[self.current]
            if crr_content_type_id == 1:
                # no more than one task_container_id of "questions" from any single user
                # so we only care for content_type_id == 0 to break loop
                user_answer_list.append(self.user_answer[self.current])
                answered_correctly_list.append(self.answered_correctly[self.current])
                self.current += 1
                continue
            if crr_user_id in added_user and ((crr_user_id != pre_added_user) or (crr_task_container_id != pre_task_container_id)):
                # known user(not prev user or differnt task container)
                return self.fix_df(user_answer_list, answered_correctly_list, pre_start)
            if len(added_user) == self.max_user:
                if  crr_user_id == pre_added_user and crr_task_container_id == pre_task_container_id:
                    user_answer_list.append(self.user_answer[self.current])
                    answered_correctly_list.append(self.answered_correctly[self.current])
                    self.current += 1
                    continue
                else:
                    return self.fix_df(user_answer_list, answered_correctly_list, pre_start)
            added_user.add(crr_user_id)
            pre_added_user = crr_user_id
            pre_task_container_id = crr_task_container_id
            user_answer_list.append(self.user_answer[self.current])
            answered_correctly_list.append(self.answered_correctly[self.current])
            self.current += 1
        if pre_start < self.current:
            return self.fix_df(user_answer_list, answered_correctly_list, pre_start)
        else:
            raise StopIteration()

In [ ]:
iter_test = Iter_Valid(validation,max_user=1000)
predicted = []
def set_predict(df):
    predicted.append(df)

In [ ]:
#https://stackoverflow.com/questions/42869495/numpy-version-of-exponential-weighted-moving-average-equivalent-to-pandas-ewm
def numpy_ewma_vectorized_v2(data, window):

    alpha = 2 /(window + 1.0)
    alpha_rev = 1-alpha
    n = data.shape[0]

    pows = alpha_rev**(np.arange(n+1))

    scale_arr = 1/pows[:-1]
    offset = data[0]*pows[1:]
    pw0 = alpha*alpha_rev**(n-1)

    mult = data*pw0*scale_arr
    cumsums = mult.cumsum()
    out = offset + cumsums*scale_arr[::-1]
    return out

In [ ]:
features_1_path = '/home/dfs/'
que_data = pd.read_pickle(features_1_path + "que_data.pickle")

questions = que_data.drop(columns=["options_number","correctness_number", "correct_answer","tags6","tags5", "tags4"]).to_dict("index")
questions1 = que_data[["tags1", "tags2", "tags3","tags4","tags5", "tags6"]].to_dict("index")

parts = que_data.part.to_dict()

In [ ]:
lec_data = pd.read_csv("/input/riiid-test-answer-prediction/lectures.csv")
lec_dict = lec_data[["lecture_id", "tag"]].set_index("lecture_id").tag.to_dict()

#Add dtype for every feature
features = [
    'task_container_id', "ts_diff_shifted", "watched","ts_diff_shifted_2",
    'content_id', "k", "el_avg", "wut", "sum_corr", "gen_mean", "u_mean", "u_mean_opp",
    'prior_question_elapsed_time', "time_diff2", "rolling_mean_5", "rolling_mean_10", "rolling_mean_15", "prior_question_had_explanation_u_part_avg",
    'prior_question_had_explanation', "hard_ratio_opp", "easy_ratio_opp", "correct_recency", "prior_question_elapsed_time_u_part_avg", "ewm_mean_10", "rolling_mean_5_prior_question",
    'last_lecture', "part_mean", "opp_mean", "mean_pause", "timestamp", "prior_part_mean",
    "container_mean", "lecs_per", "hard_ratio", "easy_ratio", 'corr_1', 'corr_2', 'corr_3', 'corr_4','corr_5', 'top_que',
    'que_count_user', 'question_repeated', "rolling_mean","time_diff3", "time_diff4",
    'user_mean', "time_diff1", "time_diff", "sessions", "session_count", "prior_question_had_explanation_ratio"
] + que_data.columns.tolist()[:-1]

features.remove("options_number")
features.remove("correct_answer")
features.remove("tags6")
features.remove("tags5")
features.remove("tags4")


test_cols = ['row_id', 'timestamp', 'user_id', 'content_id', 'content_type_id',
       'task_container_id', 'user_answer', 'answered_correctly',
       'prior_question_elapsed_time', 'prior_question_had_explanation', "st_preds","lgb_preds",
       'prior_group_responses', 'prior_group_answers_correct']

In [ ]:
k_size = 20

cols = {test_cols[k]:k for k in range(len(test_cols))}
features_dict = {features[k]:k for k in range(len(features))}

tmp_data = []

In [ ]:
import numpy as np
import ast
import copy
from collections import Counter

new_user = {'count': 0, 'mean_acc':0.5, 'correct_count': 0, 'last_lec':0, 'tmp':0,"first_timestamp":0, "second_timestamp":0,
            "third_timestamp":0, "fourth_timestamp":0, "fifth_timestamp":0, "lecs_n":0,"interaction_n":0, "ts_diff_shifted":0.,
            "part_corr":np.zeros((7), dtype=np.uint16), "part_count":np.zeros((7), dtype=np.uint16), "hard_ct":0, "hard_cr":0, "easy_ct":0, "easy_cr":0,
           "sessions":0, "session_count":0, "sum_pauses":0., "had_exp":0, "el_sum":0, "part_et": np.zeros((7), dtype=np.float), 
            "part_explan": np.zeros((7), dtype=np.uint16), "k_count": np.zeros((k_size), dtype=np.uint16), "k_corr": np.zeros((k_size), dtype=np.uint16), 
            "recent_corr":0, "priors_5": [], "ts_diff_shifted_2":0., "count_2":0, "part_count_2":np.zeros((7), dtype=np.uint16), "last_part":-1}

In [ ]:
def get_meta_data(data_1):
    
    user_id = data_1[cols['user_id']]
    content_type_id = data_1[cols['content_type_id']]
    content_id = data_1[cols['content_id']]
    prior_group_answers_correct = data_1[cols['prior_group_answers_correct']]
    timestamp = data_1[cols['timestamp']]
    task_container_id = data_1[cols['task_container_id']]
    prior_question_had_explanation = data_1[cols['prior_question_had_explanation']]
    elapsed = data_1[cols['prior_question_elapsed_time']]
    
    return user_id, content_type_id, content_id, prior_group_answers_correct, timestamp,task_container_id,prior_question_had_explanation, elapsed

In [ ]:
def add_user(user_id): 
    user_info[user_id] = copy.deepcopy(new_user)
    repeated_que_count[user_id] = {}
    groups[user_id] = []
    top_que_history[user_id] = {}

In [ ]:
def update_user_part_acc(user_id, question, answered_correctly, elapsed, explan):
    
    part = parts.get(question, -1)
    
    user_info[user_id]["part_count"][part-1] += 1
    user_info[user_id]["part_corr"][part-1] += answered_correctly

In [ ]:
def update_user(user_id, had_exp, elapsed, content_id ,answered_correctly, timestamp):
        
    user_info[user_id]['count'] += 1

    if repeated_que_count[user_id].get(content_id,  -1) == -1: #If first time question for the user
        repeated_que_count[user_id][content_id] = 1
    else:
        repeated_que_count[user_id][content_id] += 1

    if answered_correctly:
        user_info[user_id]['correct_count'] += 1

    user_info[user_id]['mean_acc'] = user_info[user_id]['correct_count']/user_info[user_id]['count']
    
    
    update_user_part_acc(user_id, content_id, answered_correctly, elapsed, had_exp)
    
    if hard_questions.get(content_id, False):
        user_info[user_id]["hard_ct"] += 1
        user_info[user_id]["hard_cr"] += answered_correctly
        
    if easy_questions.get(content_id, False):
        user_info[user_id]["easy_ct"] += 1
        user_info[user_id]["easy_cr"] += answered_correctly     
        
    
    k = que_2_k[content_id]
    user_info[user_id]["k_count"][k] += 1
    user_info[user_id]["k_corr"][k] += answered_correctly
    
    if answered_correctly:
        user_info[user_id]["recent_corr"] = timestamp
        
    if answered_correctly and content_id in que_corr:
        top_que_history[user_id][content_id] = True

In [ ]:
def update_lec_data(user_id, content_id):
    
    if watched_tags.get(str(user_id), -1) == -1:  #If user's first lecture
        watched_tags[str(user_id)] = {}
        
    if user_info.get(user_id, -1) == -1: #In case first action user does is watching a lecture
        add_user(user_id)
        
        
    user_info[user_id]["lecs_n"] += 1
    
    lec_tag  = lec_dict[content_id]

    watched_tags[str(user_id)][str(lec_tag)] = 1
    user_info[user_id]['last_lec'] = content_id

In [ ]:
def non_lag_update(user_id, timestamp, elapsed, explan, lec):
    
    
    timestamp = timestamp/8.64e+7
    
    diff_timestamp_1 = timestamp - user_info[user_id]["first_timestamp"]
    diff_timestamp_2 = timestamp - user_info[user_id]["second_timestamp"]
    diff_timestamp_3 = timestamp - user_info[user_id]["third_timestamp"]
    diff_timestamp_4 = timestamp - user_info[user_id]["fourth_timestamp"]
    diff_timestamp_5 = timestamp - user_info[user_id]["fifth_timestamp"]

    user_info[user_id]["fifth_timestamp"] = user_info[user_id]["fourth_timestamp"]
    user_info[user_id]["fourth_timestamp"] = user_info[user_id]["third_timestamp"]
    user_info[user_id]["third_timestamp"] = user_info[user_id]["second_timestamp"]
    user_info[user_id]["second_timestamp"] = user_info[user_id]["first_timestamp"]
    user_info[user_id]["first_timestamp"] = timestamp
    
    if (user_info[user_id]["second_timestamp"] - user_info[user_id]["third_timestamp"]) != 0:
        user_info[user_id]["ts_diff_shifted"] = user_info[user_id]["second_timestamp"]*8.64e+7 - user_info[user_id]["third_timestamp"]*8.64e+7
    
    if (user_info[user_id]["second_timestamp"] - user_info[user_id]["fourth_timestamp"]) != 0:
        user_info[user_id]["ts_diff_shifted_2"] = user_info[user_id]["second_timestamp"]*8.64e+7 - user_info[user_id]["fourth_timestamp"]*8.64e+7
        
        
    if not isinstance(explan, pd._libs.missing.NAType) and explan == explan and not lec:
        user_info[user_id]["priors_5"].append(explan) 
    
    if diff_timestamp_1 > 0.083:
        user_info[user_id]["sessions"] += 1
        user_info[user_id]["session_count"] = 0
        user_info[user_id]["sum_pauses"] += diff_timestamp_1
        
    user_info[user_id]["session_count"] += 1
    
    
    return  diff_timestamp_1, diff_timestamp_2, diff_timestamp_3, diff_timestamp_4, diff_timestamp_5

In [ ]:
def update_data(prior_group_answers_correct):  #Update data retrospectively
    
    global tmp_data
    
    arr = np.array(ast.literal_eval(prior_group_answers_correct))

    for i, line in enumerate(tmp_data): #Loop through users with correct answers
        
        user_id = line[cols['user_id']]
        content_type_id = line[cols['content_type_id']]
        content_id = line[cols['content_id']]
        timestamp = line[cols['timestamp']]
        task_container_id = line[cols['task_container_id']]
        
        explan = line[cols['prior_question_had_explanation']]
        if isinstance(explan, pd._libs.missing.NAType) or explan == explan:
            explan = 0
            
        elapsed = line[cols['prior_question_elapsed_time']]
        if isinstance(elapsed, pd._libs.missing.NAType) or elapsed == elapsed:
            elapsed = 0
            
        answered_correctly = arr[i]
        
        
        user_arr = groups[user_id]
        user_arr.insert(len(user_arr), answered_correctly)
        groups[user_id] = user_arr

        if content_type_id == False: #If question

            if user_info.get(user_id, -1) == -1: #If user is new
                add_user(user_id)
                
            update_user(user_id, explan, elapsed, content_id, answered_correctly, timestamp)

     
    tmp_data = []  #Flush tmp_data

In [ ]:
tmp_data = []

In [ ]:
def preprocess_1(chunk): #Optimized preprocess function for small batches, for batches > 2500 preprocess is better. Includes adding lecture
    
    
    data_1 = chunk.values
    out = np.zeros((data_1.shape[0], len(features)))
    
    batch_counts = data_1[:, [cols["user_id"],cols["content_type_id"]]]
    batch_counts = Counter(batch_counts[batch_counts[:, 1] == False][:, 0])

    global tmp_data


    for i in range(data_1.shape[0]):

        user_id, content_type_id, content_id ,prior_group_answers_correct, timestamp, task_container_id, explan, elapsed = get_meta_data(data_1[i])
        part = parts.get(content_id, -1)
        task_count = batch_counts[user_id]
        
        #When users answers are provided, update user states
        if prior_group_answers_correct == prior_group_answers_correct and prior_group_answers_correct != '[]': #If user's previous responses are there
            update_data(prior_group_answers_correct)

        tmp_data.append(data_1[i].tolist()) #Append incoming data
        
        

        if content_type_id: #If lecture, update the lecture state
            update_lec_data(user_id, content_id)

            
        #Timestamp difference
        if user_info.get(user_id, -1) == -1: #If user is new
            add_user(user_id)        
            
        
        user_info[user_id]["interaction_n"] += 1


        _ = non_lag_update(user_id, timestamp, elapsed, explan, content_type_id)
        diff_timestamp_1, diff_timestamp_2, diff_timestamp_3, diff_timestamp_4, diff_timestamp_5 = _

        if not content_type_id:
            if not isinstance(elapsed, pd._libs.missing.NAType) and elapsed == elapsed:
                user_info[user_id]["el_sum"] += elapsed
            else:
                elapsed = 0

            if isinstance(explan, pd._libs.missing.NAType) or explan != explan:
                explan = 0 
            else:
                user_info[user_id]["had_exp"] += explan

            if user_info[user_id]["count_2"] != 0 and user_info[user_id]["last_part"] != -1:
                last_part = user_info[user_id]["last_part"]
                user_info[user_id]["part_et"][last_part-1] += elapsed
                user_info[user_id]["part_explan"][last_part-1] += explan
            
            user_info[user_id]["last_part"] = part
            
            user_info[user_id]["part_count_2"][part-1] += 1
            user_info[user_id]["count_2"] += 1

            #-----Filling the array -------------------------------------------->

            out[i, features_dict['content_id']] = content_id
            out[i, features_dict['task_container_id']] = data_1[i, cols['task_container_id']]


            #prior_question_elapsed_time, and prior_question_elapsed_time
            out[i, features_dict['prior_question_elapsed_time']] = elapsed
            out[i, features_dict['el_avg']] = (user_info[user_id]["el_sum"]/(user_info[user_id]["count_2"]))/1000
            
            if user_info[user_id]["part_count_2"][part-1] != 1:
                out[i, features_dict['prior_question_elapsed_time_u_part_avg']] = (user_info[user_id]["part_et"][part-1])/(user_info[user_id]["part_count_2"][part-1]-1)

            out[i, features_dict['prior_question_had_explanation']] = explan
            
            if user_info[user_id]["part_count_2"][part-1] != 1:
                out[i, features_dict['prior_question_had_explanation_u_part_avg']] = user_info[user_id]["part_explan"][part-1]/(user_info[user_id]["part_count_2"][part-1]-1)


            #out[i, features_dict["content_type_id"]] = content_type_id
            #Fill all question features
            out[i, features_dict['bundle_id']:] = np.array(list(questions.get(content_id).values())) #No unkown question expected

            #que_count_user, question_repeated, user_mean, correct_count
            out[i, features_dict['que_count_user']] = user_info.get(user_id, {}).get('count',0)
            out[i, features_dict['question_repeated']] = repeated_que_count.get(user_id, {}).get(content_id, 0) + 1


            m = user_info.get(user_id, {}).get('mean_acc', 0)
            if m == 0: #Usually when user is new, mean drops to zero
                out[i, features_dict['user_mean']] = 0.55
            else:
                out[i, features_dict['user_mean']] = m

            out[i, features_dict['opp_mean']] = 1 - out[i, features_dict['user_mean']]

            #out[i, features_dict['correct_count']] = user_info.get(user_id, {}).get('correct_count',0)
            out[i, features_dict['last_lecture']] = user_info.get(user_id, {}).get('last_lec',0)


            #Time gap
            out[i, features_dict['time_diff']] = diff_timestamp_1
            out[i, features_dict['time_diff1']] = diff_timestamp_2
            out[i, features_dict['time_diff2']] = diff_timestamp_3
            out[i, features_dict['time_diff3']] = diff_timestamp_4
            out[i, features_dict['time_diff4']] = diff_timestamp_5

            out[i, features_dict['timestamp']] = timestamp/8.64e+7
            out[i, features_dict['correct_recency']] = (timestamp - user_info[user_id]["recent_corr"])/8.64e+7

            kk = user_info[user_id]["priors_5"][-5:]
            if len(kk) != 0: #If array not empty
                #print(str(len(kk)) + " " + str(np.array(kk).mean()))
                out[i, features_dict['rolling_mean_5_prior_question']] = np.array(kk).mean()

            out[i, features_dict['ts_diff_shifted']] = user_info[user_id]["ts_diff_shifted"]
            out[i, features_dict['ts_diff_shifted_2']] = user_info[user_id]["ts_diff_shifted_2"]

            #Container mean
            out[i, features_dict['container_mean']] = containers_mean[task_container_id]
            out[i, features_dict['lecs_per']] = user_info[user_id]["lecs_n"]/user_info[user_id]["interaction_n"]*100
            out[i, features_dict['sessions']] = user_info[user_id]["sessions"]
            out[i, features_dict['session_count']] = user_info[user_id]["session_count"]

            if user_info[user_id]["count"] != 0:
                out[i, features_dict['prior_question_had_explanation_ratio']] = user_info[user_id]["had_exp"]/(user_info[user_id]["count_2"])

            if user_info[user_id]["sessions"] != 0:
                out[i, features_dict['mean_pause']] = user_info[user_id]["sum_pauses"]/user_info[user_id]["sessions"]


            #Easy ratio
            nn = user_info.get(user_id, {}).get('easy_ct',0)
            if nn != 0:
                out[i, features_dict['easy_ratio']] = user_info.get(user_id, {}).get('easy_cr',0)/nn

            out[i, features_dict['easy_ratio_opp']] = 1 - out[i, features_dict['easy_ratio']]


            #Hard ratio
            nn = user_info.get(user_id, {}).get('hard_ct',0)
            if nn != 0:
                out[i, features_dict['hard_ratio']] = user_info.get(user_id, {}).get('hard_cr',0)/nn

            out[i, features_dict['hard_ratio_opp']] = 1 - out[i, features_dict['hard_ratio']]



            nn = user_info.get(user_id, {}).get('part_count',[])
            if nn != []: #User exists
                part = int(out[i, features_dict['part']] - 1)
                ct = nn[part]
                cr = user_info[user_id]['part_corr'][part]

                if ct != 0:
                    out[i, features_dict['part_mean']] = cr/ct


            #Rolling mean
            if groups.get(user_id, -1) != -1 and groups[user_id] != []:


                ##NOTE: this feature was suppose to do ewm with span 5 and 10 to answers without lec, however due to a bug
                ##in the generation of features, the lectures were in, and the model was trained by that, to be considered
                #if it requires fixing for next training. PS: add lag to all features when training.
                #k_acc needs to be shifted ["user_id","k"] axis, add lag to it as well
                #also mind the elapsed time and explan are wrong because they are prior, they need to be shifted

                last_arr = np.array(groups[user_id][-40:])

                out[i, features_dict['rolling_mean']] = numpy_ewma_vectorized_v2(np.array(last_arr[-30:]),5)[-1]  #Limit the length of groups array
                
                
                last_arr = last_arr[last_arr != -1]  #Remove lecture data
                out[i, features_dict['ewm_mean_10']] = numpy_ewma_vectorized_v2(np.array(last_arr[-30:]),5)[-1]

                out[i, features_dict['rolling_mean_10']] = np.array(last_arr[-10:]).mean()
                out[i, features_dict['rolling_mean_15']] = np.array(last_arr[-15:]).mean()
                out[i, features_dict['rolling_mean_5']] = np.array(last_arr[-5:]).mean()


            out[i, features_dict['prior_part_mean']] = prior_part_mean_dict[int(out[i, features_dict['part']])]

            k = que_2_k[content_id]
            out[i, features_dict['k']] = k     
            out[i, features_dict['wut']] = user_info[user_id]["ts_diff_shifted"] - elapsed*task_count
                
              
        
            correlated = que_corr.get(content_id, -1)
            nn = 0
            if correlated != -1: #If this question among top questions
                
                out[i, features_dict['top_que']] = 1
                
                corr_keys = list(correlated.keys())
                user_history = top_que_history.get(user_id, {})

                if len(user_history) != 0:  #If user answered top questions, and if user is not new
                    
                    for j in range(len(corr_keys)):

                        if top_que_history[user_id].get(corr_keys[j], False):   #If user answered this question
                            out[i, features_dict['corr_'+str(j+1)]] = correlated[corr_keys[j]]   #Set the value to the degree of the correlation
                            nn += correlated[corr_keys[j]]
                            
                            
            tmp_m = out[i, features_dict['user_mean']]
            out[i, features_dict["gen_mean"]] = tmp_m - 0.657
            out[i, features_dict["u_mean"]] = tmp_m - out[i, features_dict['que_correct_per']]
            out[i, features_dict["u_mean_opp"]] = tmp_m - out[i, features_dict['que_false_per']]
            out[i, features_dict["sum_corr"]] = nn
            
             
            #Watched_n
            usr = watched_tags.get(str(user_id), -1)
            if usr != -1:

                nn = 0

                for k in range(6):
                    nn += usr.get(str(int(questions1[content_id]["tags"+str(k+1)])), 0) 

                out[i, features_dict['watched']] = nn



    return out


In [ ]:
import lightgbm as lgb

model = lgb.Booster(model_file='/home/final_model/lgb_classifier.txt')
#stacked_model = lgb.Booster(model_file="/home/final_model/lgb_stack.txt")

In [ ]:
prior_part_mean_dict = {1: 22166.159642501425,
 2: 18714.69673913695,
 3: 23620.317746179924,
 4: 23762.753651169547,
 5: 25094.620302855932,
 6: 32417.37918735745,
 7: 47444.16407400242}

In [ ]:
user_ids = []
preds = []

In [ ]:
from tqdm import tqdm

pbar = tqdm(total=890399, position=0, leave=True)
for (current_test, current_prediction_df) in iter_test:

    test_transform = preprocess_1(current_test)
    preds.extend(model.predict(test_transform))
    user_ids.extend(current_test.user_id.tolist())
    
    pbar.update(len(current_test))

In [ ]:
from sklearn.metrics import roc_auc_score

pd.set_option("display.max_columns", 100)

In [ ]:
created = pd.DataFrame(created, columns=features)
created["user_id"] = user_ids

In [ ]:
tested_val = tested_val.iloc[:len(user_ids)]

In [ ]:
#Do this with normal model
roc_auc_score(tested_val["answered_correctly"], model.predict(tested_val[features]))

In [ ]:
roc_auc_score(tested_val["answered_correctly"], model.predict(created[features]))

In [ ]:
#Good strategy to figure which features you messed up
gains = []

for i in tqdm(features, position=0, leave=True):
    
    tmp = created.copy()
    tmp[i] = tested_val[i].reset_index(drop=True)
    
    gain = roc_auc_score(tested_val["answered_correctly"], model.predict(tmp[features]))
    
    gains.append((gain, i))

In [ ]:
gains = sorted(gains, key=lambda x: x[0], reverse=True)

In [ ]:
for i, gain in gains:  
    print("%s  == > ROC : %f" %(gain, i))